# Minimal Char level[one Gram] LM, that could Genarate Names

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt


In [54]:
names = open('names.txt', 'r').read().splitlines()
names[:10]



['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [55]:
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocabSiz = len(stoi)
print(vocabSiz)
print(itos )
print(stoi)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [56]:
random.seed(42)
random.shuffle(names)

In [57]:
blockSiz = 8
def buildDataset(names):
    X, Y = [ ], [ ]
    for n in names:
        contxtVec = [0] * blockSiz
        for char in n + '.':
            ix = stoi[char]
            X.append(contxtVec)
            Y.append(ix)
            contxtVec = contxtVec[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

xTrain, yTrain = buildDataset(names[:n1])
xDev, yDev = buildDataset(names[n1:n2])
xTest, yTest= buildDataset(names[n1:])

# While Coding Face this Error:

   * **"TypeError: cannot unpack non-iterable NoneType object"**: 

       + suggests that buildDatset(names[:n1]) is returning None instead of a tuple that can be unpacked into xTrain and yTrain.
       .... which make sence, Cos i forgot to return X, Y,,, 

In [30]:
for x, y in zip(xTrain[:20], yTrain[:20]):
    print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])


........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


# Layers

In [58]:
torch.manual_seed(42); # seed rng for reproducibility

+ ix is a tensor containing random integers, 
which are used as indices to sample batchSiz random elements from xTrain and yTrain.

+ torch.randint(0, xTrain[0].shape, (batchSiz,)) 
generates random indices within the range [0, xTrain[0].shape). Here, xTrain[0].
shape is the number of available samples.

+ xb, yb = xTrain[ix], yTrain[ix] then selects a mini-batch of inputs xb and targets yb based on these indices.

In [ ]:

#model = Sequential([
#  Embedding(vocab_size, n_embd),
#  FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
#  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
#  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
#  Linear(n_hidden, vocab_size),
#])

"""
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.model = nn.Sequential(
            nn.Embedding(vocabSiz, nEmb),
            nn.Flatten(start_dim=1),
            
            nn.Linear(80, nHidden, bias=False),
            nn.BatchNorm1d(nHidden),
            nn.Tanh(),
            
            nn.Flatten(start_dim=1),
            nn.Linear(nHidden, nHidden, bias=False),
            nn.BatchNorm1d(nHidden),
            nn.Tanh(),

            nn.Flatten(start_dim=1),
            nn.Linear(nHidden , nHidden, bias=False),
            nn.BatchNorm1d(nHidden),
            nn.Tanh(),

            nn.Linear(nHidden, vocabSiz)
        )
    
    def forward(self, x):
        return self.model(x)

model = Model()
para = model.parameters()

with torch.no_grad():
    model.model[-1].weight *= 0.1

print(sum(para.nelement() for para in model.parameters() ))
"""

24807


In [ ]:
blockSiz = 8
nEmb = 10
nHidden = 86
class BatchNorm1d(nn.Module):
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        super(BatchNorm1d, self).__init__()
        self.eps = eps
        self.momentum = momentum
        
        # Trainable parameters
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))
        
        # Buffers (non-trainable)
        self.register_buffer("runningMean", torch.zeros(dim))
        self.register_buffer("runningVar", torch.ones(dim))
        
    
    def forward(self, x):
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0, 1)
            
            # Calculate mean and variance for batch normalization
            xMean = x.mean(dim, keepdim=True)
            xVar = x.var(dim, keepdim=True, unbiased=False)
            
            # Update running statistics
            with torch.no_grad():
                self.runningMean = (1 - self.momentum) * self.runningMean + self.momentum * xMean.squeeze()
                self.runningVar = (1 - self.momentum) * self.runningVar + self.momentum * xVar.squeeze()
        else:
            xMean = self.runningMean
            xVar = self.runningVar
        
        # Normalize the input
        xHat = (x - xMean) / torch.sqrt(xVar + self.eps)
        out = self.gamma * xHat + self.beta
        return out


class FlattenConsecutive(nn.Module):
    def __init__(self, n):
        super(FlattenConsecutive, self).__init__()
        self.n = n
    
    def forward(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        return x

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # Embedding and Flattening
        self.emb = nn.Embedding(vocabSiz, nEmb)
        self.flatten = FlattenConsecutive(2)

        # Define Layers with updated dimension for layer1
        self.layer1 = nn.Sequential(
            nn.Linear(80, nHidden, bias=False),  # Change input dim to 80
            BatchNorm1d(dim=nHidden, eps=1e-5),
            nn.Tanh()
        )
        
        self.layer2 = nn.Sequential(
            nn.Linear(nHidden, nHidden, bias=False),
            BatchNorm1d(dim=nHidden, eps=1e-5),
            nn.Tanh()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(nHidden, nHidden, bias=False),
            BatchNorm1d(dim=nHidden, eps=1e-5),
            nn.Tanh()
        )

        self.outputLayer = nn.Linear(nHidden, vocabSiz)

        self.model = nn.Sequential(
            self.emb,
            self.flatten,
            self.layer1,
            self.layer2,
            self.layer3,
            self.outputLayer
        )
   

    def forward(self, x):
        print("Input Shape:", x.shape)
        x = self.emb(x)                       
        print("After Embedding:", x.shape)
        
        x = self.flatten(x)                   
        print("After Flattening:", x.shape)
        
        x = x.view(x.size(0), -1)             
        print("After View for Layer1:", x.shape)
        
        x = self.layer1(x)                    
        print("After Layer 1:", x.shape)
        
        x = self.layer2(x)                    
        print("After Layer 2:", x.shape)
        
        x = self.layer3(x)                    
        print("After Layer 3:", x.shape)
        
        x = self.outputLayer(x)               
        print("After Output Layer:", x.shape)
        
        return x




In [62]:
model = Model()

with torch.no_grad():
    model.model[-1].weight *= 0.1  

para = model.parameters()

print(sum(p.nelement() for p in para))

24807


In [63]:

for lyr in model.model:
    #for para in model.parameters()
    print(lyr.__class__.__name__, ':', sum(p.nelement() for p in lyr.parameters()))

Embedding : 270
FlattenConsecutive : 0
Sequential : 7052
Sequential : 7568
Sequential : 7568
Linear : 2349


In [65]:
maxStps = 200000
batchSiz = 32
lossi = []

for i in range(maxStps):
    ix = torch.randint(0, xTrain.shape[0], (batchSiz,))
    xb, yb = xTrain[ix], yTrain[ix]

    logits = model(xb)
    loss = F.cross_entropy(logits, yb)

    for p in para:
        p.grad = None
    loss.backward()

    for p in para:
        p.requires_grad = True
    
    lr = 0.1 if i < 15000 else 0.01
    for p in para:
        p.data += -lr * p.grad
    
    if i % 10000 == 0:
        print(f'{i:7d} / {maxStps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    break

Input Shape: torch.Size([32, 8])
After Embedding: torch.Size([32, 8, 10])
After Flattening: torch.Size([32, 4, 20])
After View for Layer1: torch.Size([32, 80])
After Layer 1: torch.Size([32, 86])
After Layer 2: torch.Size([32, 86])
After Layer 3: torch.Size([32, 86])
After Output Layer: torch.Size([32, 27])
      0 /  200000: 3.3421
